In [1]:
import itertools
import time
import permutationtest
import numpy as np
from ipyparallel import Client
from lexicographic_combinations import nth_combination, choose
from functools import reduce
import operator as op

In [11]:
SUBJECTS_IN_SUBSET = 18 # 13 values in SUBSET, 26-13=13 values in SUBSET_REMAINDER
SUBJECTS_IN_WHOLESET = 36
GIVEN_MEAN = 10.0
NUM_THREADS = 12

PERFORM_MASK_VALIDATION = False

In [12]:
if PERFORM_MASK_VALIDATION:
    precalculated_gosper_boundaries_test = permutationtest.raw_gosper_bitmask(SUBJECTS_IN_WHOLESET, SUBJECTS_IN_SUBSET, NUM_THREADS)
precalculated_gosper_boundaries = []
nchoosek = int(choose(SUBJECTS_IN_WHOLESET, SUBJECTS_IN_SUBSET))
nchoosek_interval = nchoosek // NUM_THREADS
boundary = nchoosek-1 # indices from 0 to nchoosek-1, start from last index and loop down to match reverse lexicographical order in C implementation

for i in range(0, NUM_THREADS):
    if (boundary < 0):
        break

    combo = nth_combination(SUBJECTS_IN_WHOLESET, SUBJECTS_IN_SUBSET, boundary)  # boundary+5 returns list of element numbers (len=SUBJECTS_IN_SUBSET), possible element values from 0 to SUBJECTS_IN_WHOLESET-1
    mask = 0
    for item in combo:
        mask = (1 << (SUBJECTS_IN_WHOLESET-1 - item)) | mask # reverse bitmask order to match C implementation of Gosper's hack (SUBJECTS_IN_WHOLESET-1 - item)
    
    precalculated_gosper_boundaries.append(mask)
    boundary -= nchoosek_interval

#precalculated_gosper_boundaries_test[0].reverse()

In [13]:
print(["{0:b}".format(x) for x in precalculated_gosper_boundaries])
print("Gosper boundaries from combinatorial number system: interval, total: " + str(nchoosek_interval) + ", " + str(nchoosek))

if PERFORM_MASK_VALIDATION:
    print(str(["{0:b}".format(x) for x in precalculated_gosper_boundaries_test[0]]))
    print("Gosper boundaries from gosper for verification: interval, total: " + str(precalculated_gosper_boundaries_test[1]) + ", " + str(precalculated_gosper_boundaries_test[2]))

print("Number of jobs: " + str(len(precalculated_gosper_boundaries)))

if PERFORM_MASK_VALIDATION:
    everything_same = True
    for i in range(0, len(precalculated_gosper_boundaries)):
        if (precalculated_gosper_boundaries[i] != precalculated_gosper_boundaries_test[0][i]):
            everything_same = False

    print("\nAll masks valid: " + str(everything_same))

    assert everything_same

['111111111111111111', '110001011100110101110011110010100', '1011011010110000000011101110011110', '10000100111111101001001101101000011', '10101101100101100101000011111001100', '11010101100101110000010111101011000', '100000000000000000011111111111111111', '100101010011010001111101000010101011', '101010010011010011010111100000110101', '101111011000000010110110010011000111', '110100100101001111111100010001100010', '111001110100011001010001100001101101']
Gosper boundaries from combinatorial number system: interval, total: 756261275, 9075135300
Number of jobs: 12


In [14]:
c = Client()
v = c.load_balanced_view()

In [15]:
a = [float(x) for x in range(0, SUBJECTS_IN_WHOLESET)]
print(a)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0]


In [ ]:
def calculate_combinations():
    i = 0
    for x in itertools.combinations(a, 13):
        i += 1

    return i

start_time = time.time()
print(str(calculate_combinations()))

print(str(time.time() - start_time))

In [ ]:
def gen_masks(A_len,K):
    masks = []
    num_combinations = 0
    N = A_len

    # iterate over subsets of size K
    mask = (1<<K)-1     # 2^K - 1 is always a number having exactly K 1 bits
    while mask < (1<<N):
        masks.append(mask)
 
        # catch special case
        if mask == 0:
            break
 
        # determine next mask with Gosper's hack
        a = mask & -mask                # determine rightmost 1 bit
        b = mask + a                    # determine carry bit
        mask = int(((mask^b)>>2)/a) | b # produce block of ones that begins at the least-significant bit
        
        num_combinations += 1

    return (num_combinations, masks)

for i in range(25, 35)
    

In [8]:
def run_remote_permutationtest(args):
    import permutationtest
    result = permutationtest.permutation_test_parallel(args[0], args[1], args[2], args[3], args[4])
    return result

In [16]:
# multithreaded version
start_time = time.time()
np_A = np.array(a, np.float64)
extreme_teststatistic_count = 0
count_combinations_all = 0

list_of_permutationtest_inputs = []

for i in range(0, len(precalculated_gosper_boundaries)): # iterate over predetermined bitmasks
    mask_start = precalculated_gosper_boundaries[i]
    if i + 1 < len(precalculated_gosper_boundaries):
        mask_end = precalculated_gosper_boundaries[i+1]
    else:
        mask_end = 1<<len(np_A)
    list_of_permutationtest_inputs.append((np_A, SUBJECTS_IN_SUBSET, GIVEN_MEAN, mask_start, mask_end))


test_results = v.map(run_remote_permutationtest, list_of_permutationtest_inputs)

for result in test_results:
    extreme_teststatistic_count += result[0]
    count_combinations_all += result[1]

print("(extreme_teststatistic_count, count_combinations_all): (" + str(extreme_teststatistic_count) + ", "
      + str(count_combinations_all) + ")")
print("p-value: " + str(extreme_teststatistic_count / count_combinations_all))

print("time taken: " + str("%.2f" % round(time.time() - start_time,2)) + " seconds.")

(extreme_teststatistic_count, count_combinations_all): (33788526, 9075135300)
p-value: 0.0037231980442208944
time taken: 112.77 seconds.


In [17]:
# single-threaded version
start_time = time.time()
np_A_singlethreaded = np.array(a, np.float64)
singlethreaded_result = permutationtest.permutation_test(np_A_singlethreaded, SUBJECTS_IN_SUBSET, GIVEN_MEAN)
print(singlethreaded_result)
print(singlethreaded_result[0] / singlethreaded_result[1])
print("time taken: " + str("%.2f" % round(time.time() - start_time,2)) + " seconds.")

(33788526, 9075135300)
0.0037231980442208944
time taken: 710.05 seconds.
